<h1>Moosinator 10,000</h1>
<h3>Training the model</h3>


<h3>Turning cat audio into melspectrographic images for training</h3>

In [7]:
import numpy as np
import librosa as lr
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
from PIL import Image
import os

def audio_to_image(path, height=192, width=192):
    signal, sr = lr.load(path, res_type='kaiser_fast')
    hl = signal.shape[0]//(width*1.1)
    spec = lr.feature.melspectrogram(signal, n_mels=height, hop_length=int(hl))
    img = lr.power_to_db(spec)**2
    start = (img.shape[1] - width) // 2
    return img[:, start:start+width]

meow_images = []

for i in range(1,0):
    filename = 'data/cat_'+str(i)+'.wav'
    if Path('./'+filename).exists():
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            img = audio_to_image(filename)
            meow_images.append(img)
            
            plt.imshow(img)
            plt.savefig('training/cat_'+str(i)+'.png')

<h3>Building the CNN</h3>

In [27]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# image size: 432x288
height = 432
width = 288

#Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (height,width, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

<h3>Filter training images to the CNN</h3>

In [28]:
from keras.preprocessing.image import ImageDataGenerator

#os.path.join(self.pos_dir, self.pos_labels.iloc[idx, 0])

# Data Augmentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(r"./images/training/",
                                                 target_size = (height,width),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
test_set = test_datagen.flow_from_directory(r"./images/test/",
                                            target_size = (height,width),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 146 images belonging to 2 classes.
Found 9 images belonging to 2 classes.


<h3>Let's start the training!<h3>

In [29]:

print(type(training_set[0]))
print(training_set[0])
input()
answers = np.array([0,1,1,0,1])

classifier.fit(
    x=training_set,
    y=answers,
    batch_size=None,
    epochs=1,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

<class 'tuple'>
(array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.